<a href="https://colab.research.google.com/github/Laiba-saeed92/Deep_Learning_projects/blob/main/CNN_model_and_optimization_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd
print(tensorflow.__version__)

2.18.0


In [ ]:
from tensorflow.keras.datasets import fashion_mnist
(train_images, train_labels), (test_images, test_labels)= fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
train_images= train_images/255.0 #Normalizing is a form of scaling, specifically used when working with image data, especially in Convolutional Neural Networks (CNNs) or other image-based models.
test_images= test_images/255.0 #Image pixel values are typically integers from 0 to 255.Dividing by 255.0 normalizes these values to the range [0.0, 1.0].
#ANN works with numerical/tabular data so standardcaler is used but for CNN normalization is used

In [ ]:
print(train_images[0].shape)
print(len(train_images))
print(len(test_images))
print(type(train_images))

(28, 28)
60000
10000
<class 'numpy.ndarray'>


In [ ]:
#CNNs expect 4D input with shape:
#[Number of Images, Height, Width, Channels] If your images are grayscale, you need to add that extra dimension (1 channel).
#This reshape ensures that the data has the correct shape for convolution operations.
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
def build_model(hp):
  model= Sequential()
  model.add(Input(shape=(28, 28, 1)))
  model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), kernel_size=hp.Choice( 'conv_1_kernel', values=[3,5]),activation='relu')), #kerenel_size is filter size(how big) and conv_1_filter is range of filter used btw 32 to 128 with step of 16 e.g(32, 48,...128)
  model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16), kernel_size=hp.Choice( 'conv_2_kernel', values=[3,5]),activation='relu')),
  model.add(Flatten()),
  model.add(Dense(units=hp.Int('dense_1_unit', min_value=16, max_value=32, step=16), activation='relu')),
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning rate', [1e-1, 1e-2, 1e-3])), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='fashion mnist')

In [ ]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1) #Search() for the best hyperparameters by trying many models internally (each one using model.fit() inside)
#validation_slit=0.2 this only works if X_train and y_train are array-like (e.g., NumPy arrays, not tf.data.Dataset).

Trial 3 Complete [00h 24m 25s]
val_accuracy: 0.8690000176429749

Best val_accuracy So Far: 0.9073333144187927
Total elapsed time: 00h 52m 30s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
32                |128               |conv_1_filter
5                 |3                 |conv_1_kernel
64                |32                |conv_2_filter
5                 |3                 |conv_2_kernel
16                |16                |dense_1_unit
0.001             |0.001             |learning rate

Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 213s 125ms/step - accuracy: 0.7444 - loss: 0.7222 - val_accuracy: 0.8677 - val_loss: 0.3605
Epoch 2/3
1574/1688 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 0.8856 - loss: 0.3231